In [1]:

import pandas as pd
from chefboost.training import Training
import numpy as np

In [2]:


partenza=input("Inserire città di partenza")     #Inserimento dati dall'utente
arrivo=input("Inserire città di arrivo")
soglia=input("Inserire budget")
soglia=int(soglia)


decisioni=[]
df=pd.read_csv('Table1.1.csv')
no_counter=0
inutile=0
for i in range(len(df)):                                         #Qui vengono impostate le decisioni in base agli input                                                                  inseriti
    if((df.partenza[i]==partenza) & (df.Arrivo[i]==arrivo) & (df.costo[i]<soglia+35)):
        decisioni.append('Yes')
    else:
        decisioni.append('No')
        no_counter=no_counter+1    #il no_counter serve perchè voglio sapere se il mio dataframe ha tutte decisioni "No"
print(decisioni)
df.insert(8,"Decision", decisioni)
if(no_counter==df[df.columns[0]].count()):  #Se ogni riga ha come decisione "No", allora non ha senso continuare
    print("Nessun volo disponibile")
    inutile=1                           #Serve per non far partire le prossime celle
else:
#Se no scrivi su un csv il tuo dataframe con "si" e "no". Servirà per dopo perchè riprenderai il database
    df.to_csv("C:/Users/39331\PycharmProjects\optimization\Table_riempita.csv", index=False, header=True)
    no_counter=0




['Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No']


In [3]:
def Procedura_gains(df):                                        #Questa funzione trova i guadagni ogni volta che viene                                                   chiamata e riporta in uscita la colonna dal massimo information gain

    config = {'algorithm' : 'ID3'}   #Algoritmo sito
    gains = Training.findGains(df,config)
                              #è stato utile usare .pop (metodo dizionari) perchè il comando non faceva uscire un dizionario formalmente corretto)
    gains2= gains.pop('gains')

#print(gains2)
#var mi dà il valore massimo di gains2 dandomi quindi il nome della colonna a massimo information gain

    var=max(gains2, key=gains2.get)

    stringa=df[var].astype('string')
    valori=[]                          #Una volta capita la colonna a massimo information gain, devo trovare tutti i possibili valori di  quella colonna. ed è quello che farò nel prossimo for tramite il .append (metodo liste). Ecco perchè converto fino ad avere una lista qui.
    array=pd.Series(stringa).tolist()
    lunghezza=len(array)
#print (lunghezza)
#In questo for prendo tutti i valori (eliminando i duplicati) della variabile dal maggiore information gain perchè questi valori vanno presi per fare i prossimi nodi
    for i in range(lunghezza):
        if array[i] not in valori:
            valori.append(array[i])
 #in uscita abbiamo i valori secondo il quale possiamo fare le select della tabella del database secondo la variabile a massimo information gain
    valori+=[var] #Qua mi serviva prendere anche il nome della colonna dal massimo information gain (cioè var), l'ho aggiunta alla lista dei valori convertendola da stringa a lista. La conversione sta nel fare [var]
    return(valori)


In [4]:
running=0
num_iterazioni_max=5
if(inutile==1):   #Se ci sono tutti "no", non ci sono voli disponibili.
    running=num_iterazioni_max
    print("Nessun volo disponibile secondo i  parametri indicati")

else:
    x=input("Quanto ti importa la durata da 1 a 10?")
    x=int(x)
    x=x / 10
    y=input("Quanto ti importa la differenza di prezzo da 1 a 10?")
    y=int(y)
    y=y/10
    valori_finali=Procedura_gains(df)

terminale=0
richiami=0
not_equal=0
l=0
terminali=[]

while(running<num_iterazioni_max):
#print(valori_finali)
    var=valori_finali[len(valori_finali)-1] #Valori finali ha tutti i valori della colonna dal massimo information gain + il nome della colonna in ultima posizione che                                            nella prossima istruzione verrà eliminato (tanto è stato inserito nella variabile var)
    valori_finali.pop(len(valori_finali)-1)
    #print(valori_finali)

    sottotabelle=[]
    k=0
#Sto creando una lista che conterrà tutte le sottotabelle uscite fuori facendo le select dei valori.

    for i in valori_finali:

        sottotabelle.append((df[df[var]==i]).reset_index(drop=True)) #Qui riempio la lista con le tabelle ricavate dal dataframe (df) e resetto gli indici in modo che                                                               partano da 0 e si incrementino
    #print(sottotabelle)
    indici=[]

    k=0

    for i in range(len(sottotabelle)):

        for j in range(len(sottotabelle[i])):

            if("No" in sottotabelle[i].Decision[j]):     #Qui vedo quanti "no" ci sono in ogni sottotabella per capire se è terminale (non ci sono "no", da scartare(ci sono tutti "no" oppure ci sono "no" e "si" e quindi bisogna continuare

                k=k+1
        if(k==0):
            print('La tabella ' + str(i) + ' è terminale')
            #print(sottotabelle[i])                      #C'è almeno una tabella terminale se "terminale" è diverso da zero.
            terminale=terminale+1
            for z in range(0,l):
                if(sottotabelle[i].equals(terminali[z])==False):   #Va aggiunta nelle tabelle terminali ma senza duplicare.
                    not_equal=not_equal+1
            if(not_equal==l):
                terminali.append(sottotabelle[i])
               #print(sottotabelle[i])
                l=l+1
                not_equal=0
            else:
                not_equal=0


        elif(k==len(sottotabelle[i])):
            print("La tabella numero " + str(i) + ' viene esclusa.')
            k=0

        else:
            print('Va chiamata la funzione dei guadagni per la tabella ' + str(i) + '. Infatti nella sottotabella ci sono ' + str(k) + ' No')  #Else chiama funzione guadagni .Findgains

            new_df=sottotabelle[i].drop(columns=[var])
            richiami=richiami+1
            #print(new_df)

            valori_finali=Procedura_gains(new_df)


        k=0
    running=running+1


    if(terminale!=0 & richiami==0): #Se ci sono solo tabelle terminali e tabelle scartate, abbiamo già effettuato costruzione e pruning. Dobbiamo uscire dal while
        print('Costruzione terminata prima.')
        running=num_iterazioni_max
    terminale=0                     #Altrimenti resetta i parametri per la prossima iterazione
    richiami=0

    if(running==num_iterazioni_max):
        if(len(terminali)==0):            #Caso database "incoerente". Il metodo ID3 che sto utilizzando usa solo valori nominali e quindi approssima le colonne che                                      contengono numeri. Di conseguenza alcune righe del database possono sembrare uguali ai suoi occhi e di conseguenza, per come                                   ho strutturato l'algoritmo, andrebbe all'infinito perchè non trova mai modo di finire. Ecco perchè viene inserito un numero                                    massimo di interazioni.
                                         #Se al limite massimo di interazioni prendi le sottotabelle del nodo più basso ed effettua le dovute manipolazioni
            terminali=sottotabelle

    #print(running)
if(inutile==0):
    lunghez=[]
    #for i in range (l):
     #   print('\n')
      #  print(terminali[i])
       # print('\n\n')#Printo il vettore delle sottotabelle terminali, cioè con tutti Yes


    concatRes = pd.concat(terminali).drop_duplicates().reset_index()      #Concateno le sottotabelle del nodo più basso in modo da creare una base sul quale fare il                                                                           l'ottimizzazione ed il sorting finale
    #print(concatRes)
    indici=[]
   # print(len(concatRes))
    for i in range(len(concatRes)):
        if (concatRes.Decision[i]=='No'):
          concatRes.drop(i, inplace=True)          #Levo tutte le righe con "no" sugli indici


    new_concat=concatRes.drop(columns=['index'])


    index=[]
    concato=new_concat.reset_index(drop=True)     #Indici resettati in modo che partano da 0
    concato

    df3=pd.read_csv('Table_riempita.csv')         #Riprendo il dataframe iniziale

    for i in range(df3[df3.columns[0]].count()):
        for j in range(new_concat[new_concat.columns[0]].count()):
            if((df3.Compagnia[i]==concato.Compagnia[j]) & (df3.partenza[i]==concato.partenza[j]) & (df3.scalo_1[i]==concato.scalo_1[j]) & (df3.scalo_2[i]==concato.scalo_2[j]) & (df3.Arrivo[i]==concato.Arrivo[j]) & (df3.Decision[i]==concato.Decision[j])):
                index.append(i)                    #Confronto del database iniziale per verifica e ripresa dei parametri                                     numerici

    df2=df3[df3.index.isin(index)]

    if(len(concatRes)==1):                              #Se c'è una sola  riga nel dataframe finale, non bisogna fare                                                       nulla. Solo riportare il risultato
       df2=df2.drop(columns=['Decision', 'Unnamed: 0'])
       print("\n\n")
       print(df2)


    else:

        valore=[]
        df2=df2.reset_index(drop=True)

        #print(df2)
        for i in range(len(df2)):

            valore.append((x/(x+y))*df2.durata_in_minuti[i]+(y/(x+y))*(df2.costo[i]-soglia))  #All'interno dell'append troviamo la funzione obiettivo da minimizzare secondo                                                                              l'ottimizzazione discreta. In questo momento il vettore "valore" ha i risultati della                                                                      funzione obbiettivo in base ad ogni riga del database finale
        #print(valore)
        indici_sort=np.argsort(valore)                                    #Sorting in base agli elementi del vettore "valore"
        #print(indici_sort)
        finalissimo=df2.reindex(indici_sort)
        finalissimo=finalissimo.drop(columns=['Decision','Unnamed: 0'])   #Levo le colonne superflue
        print(finalissimo)    #Tabella finale, ottimizzata e sortata secondo le specifiche

Va chiamata la funzione dei guadagni per la tabella 0. Infatti nella sottotabella ci sono 1 No
La tabella numero 1 viene esclusa.
La tabella numero 2 viene esclusa.
La tabella numero 3 viene esclusa.
La tabella numero 4 viene esclusa.
La tabella numero 5 viene esclusa.
La tabella numero 6 viene esclusa.
Va chiamata la funzione dei guadagni per la tabella 0. Infatti nella sottotabella ci sono 5 No
La tabella numero 1 viene esclusa.
Va chiamata la funzione dei guadagni per la tabella 0. Infatti nella sottotabella ci sono 1 No
La tabella numero 1 viene esclusa.
La tabella numero 2 viene esclusa.
La tabella numero 3 viene esclusa.
La tabella numero 4 viene esclusa.
Va chiamata la funzione dei guadagni per la tabella 0. Infatti nella sottotabella ci sono 5 No
La tabella numero 1 viene esclusa.
Va chiamata la funzione dei guadagni per la tabella 0. Infatti nella sottotabella ci sono 1 No
La tabella numero 1 viene esclusa.
La tabella numero 2 viene esclusa.
La tabella numero 3 viene esclusa.
